In [1]:
import numpy as np

### Exact Inference

- Edges are potential functions -> p(A,B) propto phi(A,B)
- Joint probability distribution factorises into cliques potentials
- 

In [2]:
N = 10
lattice = np.reshape(np.arange(1, N*N+1), (N, N))

In [3]:
def potential_grid(beta):
    return np.exp(beta * np.eye(2))

In [28]:
def get_nearest_neighbors(i, j, N):
    # only right and below neighbours to avoid double counting
    neighbors = []
    if i > 0:
        neighbors.append((i-1, j))
    if i < N-1:
        neighbors.append((i+1, j))
    if j > 0:
        neighbors.append((i, j-1))
    if j < N-1:
        neighbors.append((i, j+1))
        
    return neighbors

In [ ]:
def ising_exact_inference(temp):
    N = 10

    # Form the potentials for each pair of neighboring variables
    potentials = potential_grid(temp)

    # Create a grid of variables
    S = np.arange(1, N*N+1).reshape((N, N))

    # store the potentials in a dictionary - keyed by the index of the factor
    phi = {}
    # Create a graph of indices for the pairwise potentials - dimensions (x1, x2, y1, y2)
    factor_graph = np.zeros((N, N, N, N), dtype=int)

    c = 0
    # Compute pairwise potentials and factor graph
    for s1 in range(1, N*N+1):
        i1, j1 = np.argwhere(S == s1)[0]
        for s2 in range(s1+1, N*N+1):
            i2, j2 = np.argwhere(S == s2)[0]
            # If the variables are neighbors store potentials in phi and index in factor graph
            if (j1 == j2 and np.abs(i1 - i2) == 1) or (i1 == i2 and np.abs(j1 - j2) == 1):
                # increment factor index and store potential
                c += 1
                phi[c] = potentials
                # build factor graph - undirected graph therefore add both directions
                factor_graph[i1, i2, j1, j2] = c
                factor_graph[i2, i1, j2, j1] = c

    # Create row potentials - product of all pairwise potentials in the row
    rowphi = [np.ones((2, 2))]
    # traverse the rows - top to bottom
    for i in range(1, N-1):
        # initialise row potential as identity matrix
        rowphi_i = np.ones((2, 2))
        # traverse the columns - left to right
        for j in range(1, N-1):
            # multiply by below neighbour potentials
            rowphi_i = np.multiply(rowphi_i, phi[factor_graph[i, i, j, j+1]])
            # multiply by right neighbour potentials
            rowphi_i = np.multiply(rowphi_i, phi[factor_graph[i, i+1, j, j]])

        # multiply by below neighbour potentials for last column 
        # (no right neighbour since last column)
        rowphi_i = np.multiply(rowphi_i, phi[factor_graph[i, i+1, N-1, N-1]])
        rowphi.append(rowphi_i)

    # Compute the row potential for the last row
    # (no below neighbour since last row)
    rowphi_N = np.ones((2, 2))
    for j in range(N-1):
        rowphi_N = np.multiply(rowphi_N, phi[factor_graph[N-1, N-1, j, j+1]])
    rowphi.append(rowphi_N)

    # sum-product message passing on the chain
    # initialise messages
    mess = rowphi[0]
    mmess = []
    # traverse the rows - top to bottom
    for i in range(1, N-1):
        # multiply by below neighbour potentials
        mess = np.dot(mess, rowphi[i])
        # normalise
        mmess.append(mess.max())
        mess /= mmess[-1]

    # Compute the partition function logZ
    logZ = np.log(mess.sum())
    logZ += np.sum(np.log(mmess))

    return logZ


# Call the function
ising_exact_inference(1)

### Gibbs Sampling

Algorithm
- sample state, x
- sample from proposal, x' (symmetric)
- compute acceptance ratio, a
- with probability a x = x'
- save x

In [263]:
def initial_state(N):
    return np.random.choice([-1, 1], size=(N, N))

In [275]:
import random
def gibbs_sampling(max_iter=10000, burn_in=1000, beta=1.0, N=10):

    # initialise the state
    state = initial_state(N)
    initial = state.copy()

    potentials = potential_grid(beta)

    samples = []
    for _ in range(max_iter):
        flip_count = 0
        for i in range(N):
            for j in range(N):
                # compute the conditional probability
                # p(x_ij = 1 | x_-ij) proportional to product of NN potentials
                nn_potentials = []
                for n in get_nearest_neighbors(i, j, N):
                    nn_potentials.append(potentials[state[i, j], state[n[0], n[1]]])
                p_tilde = np.prod(nn_potentials)
                # normalise
                p = p_tilde / (p_tilde + 1)
                # sample from the conditional distribution
                new_state = np.random.choice([-1, 1], p=[1-p, p])
                # if state flips update the state and flip count
                if new_state != state[i, j]:
                    flip_count += 1
                    state[i, j] = new_state
        # store the state
        samples.append(state.copy())
        # if no state flips in the last iteration break - fully converged
        if len(samples) > burn_in and flip_count == 0:
            break

    return samples[burn_in:], initial


In [281]:
samples, initial = gibbs_sampling(max_iter=1000, burn_in=100, beta=4, N=10)

In [282]:
def prob_distribution(samples, N=10):
    # compute the empirical distribution by counting the number of times each state is positive
    p = np.zeros((N, N))
    for sample in samples:
        for i in range(N):
            for j in range(N):
                if sample[i, j] == 1:
                    p[i, j] += 1
    # normalise
    p /= len(samples)

    return p

In [283]:
marginals = prob_distribution(samples)
joint = marginals[0,9]*marginals[9,9]
joint

1.0

### Mean-Field Approximation

Process
- Determine factorisable distribution q
- ELBO -> KL Divergence


In [103]:
def proposal_distribution(alpha):
    q = np.exp(alpha) / (np.exp(alpha) + np.exp(-alpha))
    return q

In [241]:
def compute_elbo(alpha, beta, N):

    binary_entropy = 0
    energy = 0
    for i in range(N):
        for j in range(N):
            binary_entropy += np.log(np.exp(alpha[i,j]) + np.exp(-alpha[i,j])) - alpha[i,j] * np.tanh(alpha[i,j])
            for n in get_nearest_neighbors(i, j, N):
                energy += beta * np.tanh(alpha[i,j]) * np.tanh(alpha[n[0],n[1]])

    return binary_entropy + energy

In [242]:
def update_alpha(alpha, beta, N):

    for i in range(N):
        for j in range(N):
            alpha[i,j] = 0
            for n in get_nearest_neighbors(i, j, N):
                    alpha[i,j] += beta * np.tanh(alpha[n[0],n[1]])

    return alpha

In [303]:
def coordinate_ascent(max_iter=10000, beta=1.0, N=10, tolerance=1e-6):

    alpha = 3 * np.ones((N, N))
    state_0 = initial_state(N)
    state = state_0.copy()

    # Placeholder for storing previous ELBO value
    prev_elbo = -np.inf

    iter = 0
    for _ in range(max_iter):
        iter += 1

        for i in range(N):
            for j in range(N):
                u = np.random.uniform()
                q = proposal_distribution(alpha[i,j])
                if u < q:
                    state[i,j] = 1
                else:
                    state[i,j] = -1
        # Compute ELBO
        elbo = compute_elbo(alpha, beta, N)
        
        # Check for convergence
        if np.abs(elbo - prev_elbo) == tolerance:
            break
        
        alpha = update_alpha(alpha, beta, N)
        
        # Update previous ELBO value for convergence check
        prev_elbo = elbo

    return alpha, iter

In [304]:
alpha, iter = coordinate_ascent(max_iter=10000, beta=1, N=10, tolerance=1e-10)

In [305]:
alpha, iter

(array([[1.98923475, 2.95751576, 2.98888425, 2.98921078, 2.98921407,
         2.98921407, 2.98921078, 2.98888425, 2.95751576, 1.98923475],
        [2.95751576, 3.98787418, 3.99290209, 3.99291226, 3.99291231,
         3.99291231, 3.99291226, 3.99290209, 3.98787418, 2.95751576],
        [2.98888425, 3.99290209, 3.99729076, 3.99729672, 3.99729673,
         3.99729673, 3.99729672, 3.99729076, 3.99290209, 2.98888425],
        [2.98921078, 3.99291226, 3.99729672, 3.99730268, 3.99730268,
         3.99730268, 3.99730268, 3.99729672, 3.99291226, 2.98921078],
        [2.98921407, 3.99291231, 3.99729673, 3.99730268, 3.99730269,
         3.99730269, 3.99730268, 3.99729673, 3.99291231, 2.98921407],
        [2.98921407, 3.99291231, 3.99729673, 3.99730268, 3.99730269,
         3.99730269, 3.99730268, 3.99729673, 3.99291231, 2.98921407],
        [2.98921078, 3.99291226, 3.99729672, 3.99730268, 3.99730268,
         3.99730268, 3.99730268, 3.99729672, 3.99291226, 2.98921078],
        [2.98888425, 3.9929

In [306]:
marginals = proposal_distribution(alpha)

In [307]:
joint = marginals[0,9]*marginals[9,9]
joint

0.9635965345772143